In [97]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import itertools as it
import tabulate as tb
from functools import reduce
from operator import concat
from copy import deepcopy
import math
from collections import OrderedDict


def roundBlocks(factor, ob):
    return ob + (factor - (ob % factor))


def makeEcGroups(
        blocks,
        k, m,
        direction):

    offset = len(blocks)
    res = []

    groups = (len(blocks) // k)
    steps = len(blocks) // groups
    if direction == "Horizontal":
        for b in range(0, len(blocks), steps):
            res.append(
                blocks[b:b + steps] +
                list(range(offset + 1, (offset + 1 + m))))
            offset += m
    else:
        for s in range(0, groups):
            res.append(
                blocks[s:s+groups+offset:groups] +
                list(range(offset + 1, (offset + 1 + m))))
            offset += m

    return res


def mapNodes(blocks, hosts, offset=0):
    width = getZeros(len(blocks) or BC.value)
    formatBlock = getBlockFormat(width)
    res = OrderedDict()

    for i, _ in enumerate(blocks):
        key = "node" + str((i % hosts)+1)
        res[key] = (key in res and res[key]) or []
        res[key].append(formatBlock(blocks[i]))

    return res


def getBlockFormat(width):
    return lambda b: "{b:0{fmt}}".format(b=b, fmt=width)


def getZeros(width):
    return int(math.log(width, 10) + 1)


def makeReadSeq(K, M, ecGroups, direction, layout):
    readSequence = []
    if layout == "Append":
        if direction == "Horizontal":
            readSequence = reduce(
                concat,
                [g[:K] for g in ecGroups] + [g[-M:] for g in ecGroups])
        else:
            readSequence = reduce(concat, zip(*ecGroups))
    else:
        readSequence = reduce(concat, ecGroups)

    return readSequence


def buildModel(K, M, blocks, Dir, Layout, Nodes):
    return


def main(K, M, BC, Dir, Layout, Nodes):
    # K      - The number of source blocks to encode
    # M      - The number of parity blocks produced
    # BC     - Initial block count
    # Dir    - Coding direction
    # Layout - Block layout
    # Nodes  - Nodes count
    ##

    rb = roundBlocks(Nodes, BC)                 # Calculate geometry
    blocks = list(range(1, rb + 1))             # Generate source blocks
    ecGroups = makeEcGroups(blocks, K, M, Dir)  # Calculate EC groups
    readSequence = makeReadSeq(K, M, ecGroups, Dir, Layout)

    width = getZeros(len(readSequence) or BC)
    formatBlock = getBlockFormat(width)

    print("\n")
    print(K, M, BC, Dir, Layout, Nodes)

    print("Rounded Blocks:", rb)
    print("Encoding Groups:", len(ecGroups))
    print("K + M = ", K + M)
    print("Source Blocks:", ", ".join(map(formatBlock, blocks)))
    print("Read Layout:", ", ".join(map(formatBlock, readSequence)))

    groupsTable = tb.tabulate(
        [[", ".join(map(formatBlock, g[:K])), ", ".join(map(formatBlock, g[-M:]))]
         for g in ecGroups], ["Source", "Parity"])
    print("\n", groupsTable)

    nodesTable = tb.tabulate(
        list(map(list, mapNodes(readSequence, Nodes).items())), ["Nodes", "Blocks"])
    print("\n", nodesTable)


MaxBlocks = 256
K = widgets.IntSlider(description="K", min=2, max=MaxBlocks)
M = widgets.IntSlider(description="M", min=1, max=MaxBlocks)
BC = widgets.IntSlider(description="Origina blocks count",
                       min=1, max=10000, value=10)
D = widgets.Dropdown(options=['Horizontal', 'Vertical'],
                     description="EC Coding Direction")
Layout = widgets.Dropdown(
    options=['Inline', 'Append'], description="Blocks Manifest Layout")
Nodes = widgets.IntSlider(
    description="Nodes", min=K.value + M.value, max=MaxBlocks)

ui = widgets.VBox([K, M, BC, D, Layout, Nodes])


def updateKRange(*args):
    K.max = MaxBlocks - M.value


def updateMRange(*args):
    M.max = MaxBlocks - K.value


M.observe(updateKRange, 'value')
K.observe(updateMRange, 'value')


def updateNodesRange(*args):
    Nodes.min = K.value + M.value


Nodes.observe(updateNodesRange, 'value')

out = widgets.interactive_output(
    main,
    {
        'K': K,
        'M': M,
        'BC': BC,
        'Dir': D,
        'Layout': Layout,
        'Nodes': Nodes})

display(ui, out)


Output()